In [1]:
from Player import Player
from Trainer import Trainer
from TetrisModel import TetrisModel
from Pretrainer import Pretrainer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import pickle
import glob
import time

In [2]:
piece_dim = 8
key_dim = 12
depth = 32
gamma = 0.9
lam = 0.95
temperature = 1.0

In [3]:
# pretrainer = Pretrainer(gamma=gamma)

In [4]:
# players_data = pretrainer._load_data()

In [5]:
# pretrainer._load_dset(players_data)

In [6]:
# max_len = pretrainer._max_len
max_len = 10

In [7]:
# gt_dset = pretrainer._cache_dset()

In [8]:
agent = TetrisModel(piece_dim=piece_dim,
                    key_dim=key_dim,
                    depth=depth,
                    num_heads=4,
                    num_layers=4,
                    max_length=max_len,
                    out_dim=key_dim)

In [9]:
agent_optimizer = keras.optimizers.Adam(1e-5, clipnorm=1.5)
agent.compile(optimizer=agent_optimizer)

In [10]:
logits, piece_scores, key_scores = agent((tf.random.uniform((32, 28, 10, 1)),
                                          tf.random.uniform((32, 7), minval=0, maxval=8, dtype=tf.int32),
                                          tf.random.uniform((32, max_len), minval=0, maxval=key_dim, dtype=tf.int32)), return_scores=True)
agent.summary(), tf.shape(logits), tf.shape(piece_scores), tf.shape(key_scores)

Model: "tetris_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 70, 32)              37312     
                                                                 
 seq_embedding (SeqEmbedding  multiple                 256       
 )                                                               
                                                                 
 seq_embedding_1 (SeqEmbeddi  multiple                 384       
 ng)                                                             
                                                                 
 piece_dec_0 (DecoderLayer)  multiple                  37984     
                                                                 
 piece_dec_1 (DecoderLayer)  multiple                  37984     
                                                                 
 piece_dec_2 (DecoderLayer)  multiple                 

(None,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([32, 10, 12])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4, 32,  4,  7, 70])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4, 32,  4, 10,  7])>)

In [11]:
critic = TetrisModel(piece_dim=piece_dim,
                     key_dim=key_dim,
                     depth=depth,
                     num_heads=4,
                     num_layers=4,
                     max_length=max_len,
                     out_dim=1)

In [12]:
critic_optimizer = keras.optimizers.Adam(1e-5, clipnorm=1.5)
critic.compile(optimizer=critic_optimizer)

In [13]:
values, piece_scores, key_scores = critic((tf.random.uniform((32, 28, 10, 1)),
                                           tf.random.uniform((32, 7), minval=0, maxval=8, dtype=tf.int32),
                                           tf.random.uniform((32, max_len), minval=0, maxval=key_dim, dtype=tf.int32)), return_scores=True)
critic.summary(), tf.shape(values), tf.shape(piece_scores), tf.shape(key_scores)

Model: "tetris_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_9 (Sequential)   (32, 70, 32)              37312     
                                                                 
 seq_embedding_2 (SeqEmbeddi  multiple                 256       
 ng)                                                             
                                                                 
 seq_embedding_3 (SeqEmbeddi  multiple                 384       
 ng)                                                             
                                                                 
 piece_dec_0 (DecoderLayer)  multiple                  37984     
                                                                 
 piece_dec_1 (DecoderLayer)  multiple                  37984     
                                                                 
 piece_dec_2 (DecoderLayer)  multiple               

(None,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([32, 10,  1])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4, 32,  4,  7, 70])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4, 32,  4, 10,  7])>)

In [14]:
agent_checkpoint = tf.train.Checkpoint(model=agent, optim=agent.optimizer)
agent_checkpoint.restore('agent_checkpoint/finetuned/ckpt-1')

In [15]:
critic_checkpoint = tf.train.Checkpoint(model=critic, optim=critic.optimizer)
critic_checkpoint.restore('critic_checkpoint/finetuned/ckpt-1')

In [16]:
ref_agent = TetrisModel(piece_dim=piece_dim,
                        key_dim=key_dim,
                        depth=depth,
                        num_heads=4,
                        num_layers=4,
                        max_length=max_len,
                        out_dim=key_dim)

In [17]:
logits, piece_scores, key_scores = ref_agent((tf.random.uniform((1, 28, 10, 1)),
                                              tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                                              tf.random.uniform((1, max_len), minval=0, maxval=key_dim, dtype=tf.int32)), return_scores=True)
tf.shape(logits), tf.shape(piece_scores), tf.shape(key_scores)

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 10, 12])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4,  1,  4,  7, 70])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4,  1,  4, 10,  7])>)

In [18]:
ref_checkpoint = tf.train.Checkpoint(model=ref_agent)
ref_checkpoint.restore('agent_checkpoint/pretrained/ckpt-11')

In [19]:
# epochs = 10

In [20]:
# actor_losses, critic_losses, accs = pretrainer.train(agent, critic, gt_dset, epochs)

In [21]:
%matplotlib qt

In [22]:
# plt.plot(actor_losses)
# plt.plot(critic_losses)
# plt.plot(accs)

In [23]:
# agent_checkpoint.save('agent_checkpoint/pretrained/ckpt')
# critic_checkpoint.save('critic_checkpoint/pretrained/ckpt')

In [24]:
agent_checkpoint = tf.train.Checkpoint(model=agent, optim=agent.optimizer)
agent_checkpoint_manager = tf.train.CheckpointManager(agent_checkpoint, 'agent_checkpoint/finetuned', max_to_keep=5)

In [25]:
critic_checkpoint = tf.train.Checkpoint(model=critic, optim=critic.optimizer)
critic_checkpoint_manager = tf.train.CheckpointManager(critic_checkpoint, 'critic_checkpoint/finetuned', max_to_keep=5)

In [26]:
trainer = Trainer(agent=agent,
                  critic=critic,
                  ref_model=ref_agent,
                  max_len=max_len,
                  gamma=gamma,
                  lam=lam,
                  temperature=temperature,
                  max_episode_steps=1000)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: michaelsherrick. Use `wandb login --relogin` to force relogin


In [27]:
while True:
    trainer.train(gens=100, train_steps=10, training_actor=True)
    agent_checkpoint_manager.save()
    critic_checkpoint_manager.save()

PPO Loss: -0.01	|	KL Divergence: 1.42	|	Critic Loss: 0.06	|	

KeyboardInterrupt: 

In [36]:
agent_checkpoint_manager.save()
critic_checkpoint_manager.save()

'critic_checkpoint/finetuned\\ckpt-5'

In [ ]:
episode_data = trainer.player.run_episode(agent, critic, max_steps=1000, greedy=True, renderer=trainer.renderer)

In [ ]:
episode_boards, episode_pieces, episode_inputs, episode_probs, episode_values, episode_rewards = episode_data

In [52]:
import imageio
import numpy as np
from PIL import Image

In [68]:
frames = [Image.fromarray((board[..., 0] * 255).numpy().astype(np.uint8)).resize((100, 280), Image.Resampling.NEAREST) for board in episode_boards]
imageio.mimsave('test_gif.gif', frames, duration=0.5)

In [80]:
piece_array = np.array([[[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]],
                        [[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 1, 1, 1, 1],
                         [0, 0, 0, 0, 0]],
                        [[0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 0],
                         [0, 1, 1, 1, 0],
                         [0, 0, 0, 0, 0]],
                        [[0, 0, 0, 0, 0],
                         [0, 0, 0, 1, 0],
                         [0, 1, 1, 1, 0],
                         [0, 0, 0, 0, 0]],
                        [[0, 0, 0, 0, 0],
                         [0, 1, 0, 0, 0],
                         [0, 1, 1, 1, 0],
                         [0, 0, 0, 0, 0]],
                        [[0, 0, 0, 0, 0],
                         [0, 1, 1, 0, 0],
                         [0, 0, 1, 1, 0],
                         [0, 0, 0, 0, 0]],
                        [[0, 0, 0, 0, 0],
                         [0, 0, 1, 1, 0],
                         [0, 1, 1, 0, 0],
                         [0, 0, 0, 0, 0]],
                        [[0, 0, 0, 0, 0],
                         [0, 0, 1, 1, 0],
                         [0, 0, 1, 1, 0],
                         [0, 0, 0, 0, 0]]])

In [81]:
frames = []
for board, pieces in zip(episode_boards, episode_pieces):
    board_frame = Image.fromarray((board[..., 0] * 255).numpy().astype(np.uint8)).resize((100, 280), Image.Resampling.NEAREST)
    piece_frame = Image.fromarray(np.concatenate([(piece_array[piece] * 255).astype(np.uint8) for piece in pieces], axis=0)).resize((50, 280), Image.Resampling.NEAREST)
    frames.append(Image.fromarray(np.concatenate([np.array(board_frame), np.array(piece_frame)], axis=1)))

In [82]:
imageio.mimsave('demo.gif', frames, duration=0.5)

In [30]:
episode_advantages, episode_returns = trainer._compute_gae(episode_values, episode_rewards, trainer.gamma, trainer.lam)

In [34]:
fig, ax = plt.subplots()
ax.plot(episode_returns, label='Returns')
ax.plot(episode_values, label='Values')
ax.legend()
tf.reduce_sum(episode_rewards)

<tf.Tensor: shape=(), dtype=float32, numpy=137.11002>

In [35]:
fig, ax = plt.subplots()
ax.plot(episode_rewards, label='Rewards')
ax.plot(episode_advantages, label='Advantages')
ax.legend()

In [42]:
trainer.wandb_run.finish()

critic_loss,▂▃█▁▂▃▂▂▃▂▃▃▃▂▁▃▂▃▁▁▂▂▂▃▃▂▅▁▃▁▂▂▃▂▁▃▁▂▂▁
entropy,▄▅▁▆▅▆▅▄▆▇█▇▇▇▇▇▇█▅▇▆▆▇▇▇▆▇▇▆▇▇█▆▇▇▇▆▆▇▆
kl_div,▅▄▃▄▅▃▃▅▅▄▁▃▄▂▇▆▄▆▂▄▄▅▅▃▃▅▅▆▅▆▅█▅▇█▇▆▄▄▃
ppo_loss,▇▄▅▄▄▆▆▃▅▅▂▇▅██▅▄▇▃▆▅▅▅▅▅▄▁▅▆▃▆▄▆█▅▂▃▁▃▇
reward,▂▇▅▁█▂█▅▃█▅██▇▁▄█▅▅▂█▆██▃▇▄▁▇▅██▄▆▃▇█▁▇▂
reward_per_piece,█▇▇▇████▇██▆▇████████████▁▆█▇█▇███▇▇▇▇▇▆
unclipped_proportion,▆▆▆▇▆▅▄▅▁▆▇▆█▄▆▆▇▇▅▅▆▇▅▄▆▆▆▅▇▆▇▆▅▇▇▆▆▆▅▆
critic_loss,0.18154
entropy,-0.1292
kl_div,1.70484
ppo_loss,0.03244


In [46]:
test = tf.constant([1, 0, 1, 3, 1, 2, 1, 0])

In [56]:
tf.cast(tf.reduce_sum(tf.experimental.numpy.diff(heights) ** 2), tf.float32) * 0.01

<tf.Tensor: shape=(), dtype=float32, numpy=0.61>

In [55]:
heights = trainer.player._get_heights(tf.cast(episode_boards[-1, ..., 0], tf.int32))